In [1]:
import numpy as np
import numpy as pn
import pandas as pd
import seaborn as sns
import sklearn.metrics

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from matplotlib import pyplot as plt
from src.data.create_data import *
from src.features.create_features import *

In [2]:
train_features, target = load_train_data('../data/external/train.csv')
test_features = load_test_data('../data/external/test.csv')

In [3]:
encode_ordinal_features(train_features)
freq_encode_nominal_features(train_features)
train_features

,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,...,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition
Id,,,,,,,,,,,,,,,,,,,,,
1,60,1151,65.0,8450,1454,1369,925,1311,1459,1052,...,0,0,-1,1179,1406,0,2,2008,1267,1198
2,20,1151,80.0,9600,1454,1369,925,1311,1459,47,...,0,0,-1,1179,1406,0,5,2007,1267,1198
3,60,1151,68.0,11250,1454,1369,484,1311,1459,1052,...,0,0,-1,1179,1406,0,9,2008,1267,1198
4,70,1151,60.0,9550,1454,1369,484,1311,1459,263,...,0,0,-1,1179,1406,0,2,2006,1267,101
5,60,1151,84.0,14260,1454,1369,484,1311,1459,47,...,0,0,-1,1179,1406,0,12,2008,1267,1198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,1151,62.0,7917,1454,1369,925,1311,1459,1052,...,0,0,-1,1179,1406,0,8,2007,1267,1198
1457,20,1151,85.0,13175,1454,1369,925,1311,1459,1052,...,0,0,-1,157,1406,0,2,2010,1267,1198
1458,70,1151,66.0,9042,1454,1369,925,1311,1459,1052,...,0,0,-1,59,49,2500,5,2010,1267,1198


In [4]:
encode_ordinal_features(test_features)
freq_encode_nominal_features(test_features)
test_features

,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,...,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,10,80.0,11622,1453,1352,934,1311,1457,1081,...,120,0,-1,172,1408,0,6,2010,1258,1204
1462,20,1114,81.0,14267,1453,1352,484,1311,1457,248,...,0,0,-1,1169,3,12500,6,2010,1258,1204
1463,60,1114,74.0,13830,1453,1352,484,1311,1457,1081,...,0,0,-1,172,1408,0,3,2010,1258,1204
1464,60,1114,78.0,9978,1453,1352,484,1311,1457,1081,...,0,0,-1,1169,1408,0,6,2010,1258,1204
1465,120,1114,43.0,5005,1453,1352,484,70,1457,1081,...,144,0,-1,1169,1408,0,1,2010,1258,1204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,242,21.0,1936,1453,1352,934,1311,1457,1081,...,0,0,-1,1169,1408,0,6,2006,1258,1204
2916,160,242,21.0,1894,1453,1352,934,1311,1457,1081,...,0,0,-1,1169,1408,0,4,2006,1258,89
2917,20,1114,160.0,20000,1453,1352,934,1311,1457,1081,...,0,0,-1,1169,1408,0,9,2006,1258,89


In [5]:
cat = train_features.select_dtypes(include=object).columns
disc_cols = [*cat, 'mssubclass', 'overallqual', 'overallcond']

In [6]:
derived_features(train_features)
train_features

,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,...,drivfeat1,drivfeat2,drivfeat3,drivfeat4,drivfeat5,drivfeat6,pcafeat1,pcafeat2,pcafeat3,pcafeat4
Id,,,,,,,,,,,,,,,,,,,,,
1,60,1151,65.0,91.923882,1454,1369,925,1311,1459,1052,...,35,12,12,61,259.008989,524.587356,0.000000,0,259560,6009.0
2,20,1151,80.0,97.979590,1454,1369,925,1311,1459,47,...,48,12,9,298,1.084862,524.587356,0.000000,3,47080,5928.0
3,60,1151,68.0,106.066017,1454,1369,484,1311,1459,1052,...,35,12,12,42,259.008989,524.587356,0.000000,3,259560,6003.0
4,70,1151,60.0,97.724101,1454,1369,484,1311,1459,263,...,35,12,9,307,1.084862,426.857881,0.000000,4,7828,5994.0
5,60,1151,84.0,119.415242,1454,1369,484,1311,1459,47,...,40,12,12,276,259.008989,524.587356,0.000000,3,259560,6000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,1151,62.0,88.977525,1454,1369,925,1311,1459,1052,...,30,12,9,40,1.084862,524.587356,0.000000,3,259560,5997.0
1457,20,1151,85.0,114.782403,1454,1369,925,1311,1459,1052,...,36,12,9,349,239.304688,524.587356,392.462963,6,15336,5934.0
1458,70,1151,66.0,95.089432,1454,1369,925,1311,1459,1052,...,63,12,20,60,1.084862,524.587356,0.000000,8,3660,5823.0


In [7]:
derived_features(test_features)
test_features

,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,...,drivfeat1,drivfeat2,drivfeat3,drivfeat4,drivfeat5,drivfeat6,pcafeat1,pcafeat2,pcafeat3,pcafeat4
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,10,80.0,107.805380,1453,1352,934,1311,1457,1081,...,30,9,9,260,0.541387,527.256741,315.268293,0,260100,5883.0
1462,20,1114,81.0,119.444548,1453,1352,484,1311,1457,248,...,36,9,9,429,264.403226,527.256741,0.004850,0,39770,5874.0
1463,60,1114,74.0,117.601020,1453,1352,484,1311,1457,1081,...,25,12,9,246,0.541387,527.256741,0.004850,3,260100,5991.0
1464,60,1114,78.0,99.889939,1453,1352,484,1311,1457,1081,...,36,9,9,396,264.403226,527.256741,0.004850,4,260100,5994.0
1465,120,1114,43.0,70.746025,1453,1352,484,70,1457,1081,...,40,12,12,226,0.541387,527.256741,0.004850,0,43780,5976.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,242,21.0,44.000000,1453,1352,934,1311,1457,1081,...,28,9,9,0,0.541387,0.000000,0.004850,0,4290,1.0
2916,160,242,21.0,43.520110,1453,1352,934,1311,1457,1081,...,20,9,9,24,0.541387,353.833333,0.004850,0,4290,5910.0
2917,20,1114,160.0,141.421356,1453,1352,934,1311,1457,1081,...,35,9,9,474,0.541387,411.150510,0.004850,3,260100,5880.0


In [10]:
fig, axes = plt.subplots(nrows=16, ncols=6, figsize=[36, 54], dpi=300)
axes = axes.ravel()

for i, col in enumerate(test_features.columns):
    discrete = True if train_features[col].dtype == int else False
    sns.histplot(data=train_features, x=col, discrete=discrete, ax=axes[i])

plt.show()

In [8]:
mask = []

for t in train_features.columns:
    if t in disc_cols:
        mask.append(True)
    else:
        mask.append(False)

mi_score(train_features, target, mask)

overallqual     0.565781
grlivarea       0.489995
drivfeat1       0.428658
garagecars      0.369517
yearbuilt       0.367288
                  ...   
street          0.000000
miscval         0.000000
lowqualfinsf    0.000000
utilities       0.000000
mosold          0.000000
Length: 96, dtype: float64

In [9]:
final_train = train_features.drop(['mosold', 'street', 'poolarea', '3ssnporch', 'utilities', 'landslope', 'heating'],
                                  axis=1)
final_test = test_features.drop(['mosold', 'street', 'poolarea', '3ssnporch', 'utilities', 'landslope', 'heating'],
                                axis=1)

In [10]:
trainX, validX, trainY, validY = train_test_split(final_train, target, test_size=0.2, random_state=48)
trainX.shape, validX.shape, trainY.shape, validY.shape

((1168, 89), (292, 89), (1168,), (292,))

## Tress

In [11]:
base_tree = DecisionTreeRegressor(max_depth=3, min_samples_leaf=25, min_samples_split=50)

In [62]:
kfold = KFold(n_splits=9, random_state=48, shuffle=True)

In [13]:
trees = []

for t, v in kfold.split(trainX, trainY):
    tree = base_tree.fit(trainX.iloc[t], trainY.iloc[t])
    train_score = mean_squared_error(trainY.iloc[t], tree.predict(trainX.iloc[t]), squared=False)
    valid_score = mean_squared_error(trainY.iloc[v], tree.predict(trainX.iloc[v]), squared=False)

    trees.append([tree, train_score, valid_score])

In [14]:
trees

[[DecisionTreeRegressor(max_depth=3, min_samples_leaf=25, min_samples_split=50),
  43249.228152837335,
  39270.36292816629],
 [DecisionTreeRegressor(max_depth=3, min_samples_leaf=25, min_samples_split=50),
  43786.130967429846,
  41281.22199135103],
 [DecisionTreeRegressor(max_depth=3, min_samples_leaf=25, min_samples_split=50),
  41378.13817173973,
  45749.02830371832],
 [DecisionTreeRegressor(max_depth=3, min_samples_leaf=25, min_samples_split=50),
  43288.51865747794,
  37915.22589875001],
 [DecisionTreeRegressor(max_depth=3, min_samples_leaf=25, min_samples_split=50),
  39062.46368566423,
  54023.78104399291]]

In [15]:
target = np.zeros((final_test.shape[0]))

for i in trees:
    p = i[0].predict(final_test)
    target += p

target = target / len(trees)

### optimize tree hyper-parameters

In [21]:
estimator = GradientBoostingRegressor(loss='squared_error')
paraDis = {'learning_rate': [0.1, 0.033, 0.01, 0.0033, 0.001, 0.00033],
           'max_depth': [9, 15, 21, 27, 35],
           'max_iter': [100, 150, 200, 250, 300],
           'min_samples_leaf': [25, 50, 100, 150],
           'min_samples_split': [50, 100, 200, 300],
           'min_impurity_decrease': [0.0001, 0.001, 0.01, 0.1]
           }

rgs = RandomizedSearchCV(estimator,
                         param_distributions=paraDis,
                         scoring=sklearn.metrics.make_scorer(mean_squared_error, greater_is_better=False),
                         cv=5,
                         n_jobs=-1,
                         verbose=1)

In [22]:
rgs.fit(trainX, trainY)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'max_depth': [3, 9, 15, 21],
                                        'min_impurity_decrease': [0.0001, 0.001,
                                                                  0.01, 0.1],
                                        'min_samples_leaf': [25, 50, 100, 150],
                                        'min_samples_split': [50, 100, 200,
                                                              300],
                                        'n_estimators': [100, 150, 200, 250,
                                                         300, 350]},
                   scoring=make_scorer(mean_squared_error, greater_is_better=False),
                   verbose=1)

In [23]:
rgs.best_estimator_

RandomForestRegressor(max_depth=9, min_impurity_decrease=0.0001,
                      min_samples_leaf=25, min_samples_split=50,
                      n_estimators=300)

In [72]:
paraDis = {'max_depth': [9, 10, 11, 12, 13],
           'min_samples_leaf': [15, 16, 17],
           'min_samples_split': [27, 28, 29, 30, 31],
           'n_estimators': [290]}

rgs_1 = RandomizedSearchCV(estimator,
                           param_distributions=paraDis,
                           scoring=sklearn.metrics.make_scorer(mean_squared_error, greater_is_better=False),
                           cv=5,
                           n_jobs=-1,
                           verbose=1)

In [73]:
gs = GridSearchCV(estimator,
                  param_grid=paraDis,
                  scoring=sklearn.metrics.make_scorer(mean_squared_error, greater_is_better=False),
                  cv=5,
                  n_jobs=-1,
                  verbose=1)

In [74]:
rgs_1.fit(trainX, trainY)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'max_depth': [9, 10, 11, 12, 13],
                                        'min_samples_leaf': [15, 16, 17],
                                        'min_samples_split': [27, 28, 29, 30,
                                                              31],
                                        'n_estimators': [290]},
                   scoring=make_scorer(mean_squared_error, greater_is_better=False),
                   verbose=1)

In [56]:
rgs_1.best_estimator_

RandomForestRegressor(max_depth=11, min_samples_leaf=16, min_samples_split=29,
                      n_estimators=290)

In [75]:
gs.fit(trainX, trainY)

Fitting 5 folds for each of 75 candidates, totalling 375 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [9, 10, 11, 12, 13],
                         'min_samples_leaf': [15, 16, 17],
                         'min_samples_split': [27, 28, 29, 30, 31],
                         'n_estimators': [290]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False),
             verbose=1)

In [76]:
gs.best_estimator_

RandomForestRegressor(max_depth=9, min_samples_leaf=15, min_samples_split=30,
                      n_estimators=290)

### inference

In [88]:
model =

In [89]:
models = []

for t, v in kfold.split(trainX, trainY):
    model.fit(trainX.iloc[t], trainY.iloc[t])
    train_score = mean_squared_error(trainY.iloc[t], model.predict(trainX.iloc[t]), squared=False)
    valid_score = mean_squared_error(trainY.iloc[v], model.predict(trainX.iloc[v]), squared=False)

    models.append([model, train_score, valid_score])

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    7.0s remaining:   21.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    7.6s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.4s remaining:    1.4s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.3s remaining:    1.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.9s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    8.8s remaining:   26.7s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    9.4s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Don

In [90]:
target_t = np.zeros((validX.shape[0]))

for i in models:
    p = i[0].predict(validX)
    target_t += p

target_t = target_t / 9

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.2s remaining:    0.8s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.6s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.1s remaining:    0.6s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.5s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.2s remaining:    0.8s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.6s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.2s remaining:    0.8s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.5s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Don

In [91]:
mean_squared_error(np.log(validY), np.log(target_t), squared=False)

0.15591820281461496

In [92]:
target = np.zeros((final_test.shape[0]))

for i in models:
    p = i[0].predict(final_test)
    target += p

target = target / 9
target

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.4s remaining:    1.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.7s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.3s remaining:    1.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.7s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.2s remaining:    0.9s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.7s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    0.3s remaining:    1.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.7s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Don

array([127713.20119105, 157667.41043824, 181627.66435628, ...,
       155258.89964015, 113040.23620789, 228913.69783583])

In [93]:
pd.DataFrame(target, columns=['SalePrice'], index=final_test.index).reset_index().to_csv(
    '../data/submission/submission_3_4.csv', index=False)